In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset=pd.read_csv('/content/drive/MyDrive/neural.csv')


In [ ]:
dependent_variable='label'
independent_variables=dataset.columns.tolist()
independent_variables.remove(dependent_variable)
X=dataset[independent_variables].values
Y=dataset[dependent_variable].values
X_train, X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
batch_size = 100

trainset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(Y_train))
valset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(Y_test))
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=False)
valloader= DataLoader(valset, batch_size=batch_size, shuffle=False) 


In [ ]:
input_size = 784
hidden_sizes = [200, 100]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

In [ ]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 20
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        images = images.view(images.shape[0], -1)
    
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
      
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.08747307630255818
Epoch 1 - Training loss: 0.08665395262651146
Epoch 2 - Training loss: 0.08463322226889432
Epoch 3 - Training loss: 0.08292756923474372
Epoch 4 - Training loss: 0.0820821040775627
Epoch 5 - Training loss: 0.08057637321762741
Epoch 6 - Training loss: 0.07883739760145544
Epoch 7 - Training loss: 0.07803220944479108
Epoch 8 - Training loss: 0.07717286339029669
Epoch 9 - Training loss: 0.0763358814176172
Epoch 10 - Training loss: 0.0756054432829842
Epoch 11 - Training loss: 0.07521733297035098
Epoch 12 - Training loss: 0.07434921094682068
Epoch 13 - Training loss: 0.07397966815624386
Epoch 14 - Training loss: 0.07398648359812796
Epoch 15 - Training loss: 0.07371154753491282
Epoch 16 - Training loss: 0.07364574030507356
Epoch 17 - Training loss: 0.07459187032654882
Epoch 18 - Training loss: 0.07473533970769494
Epoch 19 - Training loss: 0.0760587310884148

Training Time (in minutes) = 0.0966145634651184


In [ ]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Accuracy % =", ((correct_count/all_count)*100))

Accuracy % = 83.8
